1)docker images 列出镜像列表

lyc: docker $ docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE

ubuntu              15.10               9391ce633eb1        13 hours ago        101MB

ubuntu              16.04               9361ce633ff1        13 hours ago        118MB

ubuntu              latest              94e814e2efa8        13 hours ago        88.9MB

hello-world         latest              fce289e99eb9        2 months ago        1.84kB

同一个仓库源可以有不同的tag，使用时的格式： REPOSITORY：TAG，比如ubuntu：16.04，如果tag不指定，將默认用最新的版本

2) docker pull 拖取/下载镜像。当使用docker run运行容器时，如果本地不存在，docker会自动从docker镜像仓库中下载，默认从Docker Hub。也可以使用docker pull预先下载，如：

docker pull ubuntu:13:10

docker pull python

下载完成后，可直接运行该镜像

3） docker search 搜索镜像

docker search python

当从镜像仓库中下载的镜像不能满足要求时，可以通过以下两种方式对镜像进行更新或创建：

1. 从已经创建的容器中更新镜像，并且提交这个镜像

2. 使用dockerfile指令来创建一个新的镜像


1）更新镜像

在更新镜像前，需要使用镜像来创建一个容器：

docker run -t -i ubuntu:15.10 /bin/bash

然后，在运行的容器内使用apt-get update命令进行更新

完成操作之后，输入exit命令来退出这个容器

以上容器的id为e218edb10161，需要通过命令docker commit来提交容器副本：

docker commit -m="has update" -a="runoob" e218edb10161 runoob/ubuntu:v2

各个参数说明：

-m:提交的描述信息

-a：指定镜像作者

e218edb10161：容器的id

runoob/ubuntu:v2:指定要创建的目标镜像名

用命令docker images可以查看到新镜像runoob/ubuntu:v2

docker images

返回：

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE

runoob/ubuntu       v2                  70bf1840fd7c        15 seconds ago      158.5 MB

可以使用新镜像runoob/ubuntu:v2来启动一个容器：

docker run -t -i runoob/ubuntu:v2 /bin/bash

2)可以使用命令docker build，从零开始来创建一个新的镜像。为此，需要创建一个i哦dockerfile文件，其中包含一组指令来告诉docker如何创建镜像：

cat dockerfile

返回：

FROM    centos:6.7

MAINTAINER      Fisher "fisher@sudops.com"

RUN     /bin/echo 'root:123456' |chpasswd

RUN     useradd runoob

RUN     /bin/echo 'runoob:123456' |chpasswd

RUN     /bin/echo -e "LANG=\"en_US.UTF-8\"" >/etc/default/local

EXPOSE  2

EXPOSE  80

CMD     /usr/sbin/sshd -D


每一个指令都会在镜像上创建一个新的层，每个指令的前缀都必须是大写的，如FROM、RUN等。

第一条FROM，指定使用哪个镜像源

RUN指令告诉docker在镜像内执行命令，安装了什么。

然后，使用dockerfile文件，通过docker build命令来构建一个镜像

docker build -t runoob/centos:6.7

返回：

Sending build context to Docker daemon 17.92 kB
Step 1 : FROM centos:6.7
 ---&gt; d95b5ca17cc3
Step 2 : MAINTAINER Fisher "fisher@sudops.com"
 ---&gt; Using cache
 ---&gt; 0c92299c6f03
Step 3 : RUN /bin/echo 'root:123456' |chpasswd
 ---&gt; Using cache
 ---&gt; 0397ce2fbd0a
Step 4 : RUN useradd runoob
......

参数说明：

-t:指定要创建的目标镜像名

runoob/centos:6.7：dockerfile文件所在目录，可以指定dockerfile的绝对路径

可以使用docker images查看创建的镜像已经存在与列表中，镜像id为860c279d2fec

docker images

返回：

REPOSITORY          TAG                 IMAGE ID            CREATED              SIZE
runoob/centos       6.7                 860c279d2fec        About a minute ago   190.6 MB
runoob/ubuntu       v2                  70bf1840fd7c        17 hours ago         158.5 MB


可以使用新的镜像来创建容器

docker run -t -i runoob/centos:6.7 /bin/bash

id runoob

返回：

uid=500(runoob) gid=500(runoob) groups=500(runoob) #可以看到新镜像已经包含我们创建的用户runoob

可以使用docker tag命令，为镜像创建一个新的标签：

docker tag 860c279d2fec runoob/centos::dev

格式为： docker tag 镜像id 用户名称 镜像源名：新的标签名